In [ ]:
pip install datawig

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 59 kB/s 
     |████████████████████████████████| 7.0 MB 10.8 MB/s 
     |████████████████████████████████| 10.4 MB 51.0 MB/s 
     |████████████████████████████████| 29.6 MB 1.3 MB/s 
     |████████████████████████████████| 13.8 MB 49.9 MB/s 
     |████████████████████████████████| 38.2 MB 1.4 MB/s 
     |████████████████████████████████| 28.5 MB 62.6 MB/s 
     |████████████████████████████████| 28.5 MB 1.2 MB/s 
     |████████████████████████████████| 27.4 MB 1.2 MB/s 
     |████████████████████████████████| 27.4 MB 1.4 MB/s 
     |████████████████████████████████| 27.4 MB 61.6 MB/s 
     |████████████████████████████████| 27.4 MB 1.2 MB/s 
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
  Created wheel for datawig: filename=datawig-0.2.0-py3-none-any.whl size=72680 sha256=6070e3343899b630c7c8adc959a5c0f18f4d167ba25265e02d23f216638

In [ ]:
import datawig 
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('/content/extracted_downsampled_icu_delirium_mimic.csv')
df.columns

Index(['ALT', 'AST', 'Abdominal Girth (cm)', 'Access Pressure',
       'Admission Weight (Kg)', 'Albumin', 'Alkaline Phosphate', 'Anion gap',
       'Apnea Interval', 'Apnea Time Interval',
       ...
       'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE',
       'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'GENDER', 'AGE',
       'Delirium_'],
      dtype='object', length=319)

In [ ]:
cols=list(df)
for col in cols:
  if df[col].isnull().sum()/df.shape[0]>0.73:
    df.drop(col,axis=1,inplace=True)

In [ ]:
df.to_csv('delirium_missing_73.csv')

In [ ]:
df_p=df[df['Delirium_']==1]
df_n=df[df['Delirium_']==0]
print(len(df_n))
print(len(df_p))

3428
1714


In [ ]:
train,test=train_test_split(df,stratify=df['Delirium_'],test_size=0.2,shuffle=True,random_state=11)

In [ ]:
len(train.columns)

88

In [ ]:
#handling missing values
def mergeDFs(df,df_imputed,imputed_col):
  l=imputed_col+'_imputed_proba'
  if l not in list(df_imputed):
    df_imputed2=df_imputed.drop(imputed_col,axis=1)
  else:
    df_imputed2=df_imputed.drop([imputed_col,l],axis=1)
  df=df[df[imputed_col].isnull()==False]
  df_imputed2=df_imputed2.rename(columns = {imputed_col+'_imputed':imputed_col})
  return pd.concat([df,df_imputed2],sort=False)

def impute(df_train,df_test):
    cols=list(df_train)
    i=1
    for col in cols:
        print(i)
        missing = df_train[df_train[col].isnull()]
        missing_test=df_test[df_test[col].isnull()]
        if missing.shape[0]>0 or missing_test.shape[0]>0:
            in_cols=cols
            in_cols.remove(col)
            imputer = datawig.SimpleImputer(
            input_columns=in_cols,
            output_column=col
            )
            imputer.fit(train_df = df_train)
            imputed = imputer.predict(missing)
            imputed_test = imputer.predict(missing_test)
            df_train=mergeDFs(df_train,imputed,col)
            df_test=mergeDFs(df_test,imputed_test,col)
        i+=1
    for col in cols:
        mvals=df_train[col].isnull().sum()
        mvals_test=df_test[col].isnull().sum()
        if mvals>0 or mvals_test>0:
            df_train, df_test=impute(df_train,df_test)
    return df_train, df_test

In [ ]:
train_im,test_im=impute(train,test)

1


INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2022-11-10 15:48:24,752 [INFO]  NumExpr defaulting to 2 threads.
INFO:root:
========== start: fit model
2022-11-10 15:48:26,547 [INFO]  
========== start: fit model
2022-11-10 15:48:26,555 [WARNING]  Already bound, ignoring bind()
INFO:root:Epoch[0] Batch [0-116]	Speed: 983.38 samples/sec	cross-entropy=13.324761	ALT-accuracy=8.025641
2022-11-10 15:48:28,470 [INFO]  Epoch[0] Batch [0-116]	Speed: 983.38 samples/sec	cross-entropy=13.324761	ALT-accuracy=8.025641
INFO:root:Epoch[0] Train-cross-entropy=10.529714
2022-11-10 15:48:30,330 [INFO]  Epoch[0] Train-cross-entropy=10.529714
INFO:root:Epoch[0] Train-ALT-accuracy=8.155172
2022-11-10 15:48:30,339 [INFO]  Epoch[0] Train-ALT-accuracy=8.155172
INFO:root:Epoch[0] Time cost=3.783
2022-11-10 15:48:30,347 [INFO]  Epoch[0] Time cost=3.783
INFO:root:Saved checkpoint to "alt/model-0000.params"
2022-11-10 15:48:30,358 [INFO]  Saved checkpoint to "alt/model-0000.params"
INFO:root:Epoch[0] Validati

2
3
4


/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in greater_equal
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in less
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)


5
6
7
8
9


...

39


/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in greater_equal
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in less
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)


40
41
42
43
44
45


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


46
47
48
49


In [ ]:
train_im.columns

Index(['ALT', 'AST', 'Admission Weight (Kg)', 'Albumin', 'Alkaline Phosphate',
       'Anion gap', 'Apnea Interval', 'Arterial Base Excess',
       'Arterial Blood Pressure Alarm - High',
       'Arterial Blood Pressure Alarm - Low', 'Arterial Blood Pressure mean',
       'Arterial CO2 Pressure', 'BUN', 'Braden Score', 'CK (CPK)',
       'Calcium non-ionized', 'Central Venous Pressure', 'Chloride (serum)',
       'Creatinine', 'Daily Weight', 'Differential-Basos', 'Differential-Eos',
       'Differential-Lymphs', 'Differential-Monos', 'Differential-Neuts',
       'Eye Opening', 'Fingerstick Glucose', 'Fspn High', 'GCS Total',
       'HCO3 (serum)', 'Heart Rate', 'Hematocrit (whole blood - calc)',
       'Hemoglobin', 'INR', 'Inspiratory Time', 'Inspired O2 Fraction',
       'Ionized Calcium', 'LDH', 'Lactic Acid', 'Magnesium',
       'Mean Airway Pressure', 'Minute Volume', 'Motor Response', 'NBP Mean',
       'Non Invasive Blood Pressure mean', 'O2 Flow',
       'O2 saturation pulseox

In [ ]:
train_im.to_csv('delirium_mimic_icu_train_imputed.csv')
test_im.to_csv('delirium_mimic_icu_test_imputed.csv')

In [ ]:
from google.colab import files
files.download('delirium_mimic_icu_train_imputed.csv')
files.download('delirium_mimic_icu_test_imputed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>